<a href="https://colab.research.google.com/github/2025-01-sookmyung-opensource/final_project/blob/pre-min/MobileNetV3_amount_estimator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torchvision.models as models
#mobilenet 사전학습 모델 불러오기
mobilenet = models.mobilenet_v3_small(pretrained=True)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [2]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [32]:
import os

# 데이터 경로
data_dir = "/content/drive/MyDrive/image-20250617T154733Z-1-001/image"
q_labels = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5']

# 잘 올라갔는지 확인
print("클래스 디렉토리들:", os.listdir(data_dir))


클래스 디렉토리들: ['꽁치조림', '닭볶음탕', '닭죽', '깻잎장아찌', '녹두빈대떡', '꽃게탕', '달걀말이', '닭튀김', '닭갈비', '김치볶음밥', '김밥', '곱창전골', '감자탕', '갈비탕', '감자튀김(스틱형)', '감자튀김(웨지감자)', '갈치조림']


In [31]:
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

class QuantityDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        #self.image_paths = []
        #self.labels = []
        self.transform = transform
        self.root_dir = root_dir
        self.q_levels = ['Q1', 'Q2', 'Q3', 'Q4', 'Q5']
        #self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
        self.samples = []

        for food_name in os.listdir(root_dir):
            food_path = os.path.join(root_dir, food_name)
            if not os.path.isdir(food_path):
                continue

            for q_idx, q in enumerate(self.q_levels):
                q_path = os.path.join(food_path, q)
                if not os.path.isdir(q_path):
                    continue

                for fname in os.listdir(q_path):
                    if fname.lower().endswith(('.jpg', '.jpeg', '.png')):
                        img_path = os.path.join(q_path, fname)
                        self.samples.append((img_path, q_idx))  # Q1=0 ~ Q5=4


    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label


In [34]:
from torchvision import datasets
from torch.utils.data import DataLoader, random_split
from sklearn.model_selection import train_test_split
import torch

# 이미지 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # MobileNetV3 input size
    transforms.ToTensor(),
])

# 데이터셋 불러오기
dataset = QuantityDataset(root_dir=data_dir, transform=transform)

# 데이터 분리 (예: 80% train, 20% val)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42))
# -> generator 추가.  항상 동일하게 나누고 싶어서 선택.

# DataLoader 정의
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)



In [35]:
import torchvision.models as models
import torch.nn as nn
#사전학습된 모델 불러오기
model = models.mobilenet_v3_small(pretrained=True)
#마지막 레이어를 클래스 수에 맞게 바꾸는 코드.
model.classifier[3] = nn.Linear(model.classifier[3].in_features, 5)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Small_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Small_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [36]:
import torch
import torch.nn as nn
import torch.optim as optim

# 장치 설정 - GPU 사용 / 모델 학습 성능에 영향
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#GPU로 모델을 옮겨줌.
model = model.to(device)

# 손실 함수와 최적화기- 클래스 분류
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001) #모델의 가중치 업데이트


In [37]:
num_epochs = 2  # 원하는 만큼 조정

for epoch in range(num_epochs):
   #training 학습
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 통계
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    acc = 100 * correct / total
    #validation 검증
    model.eval()
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_acc = 100 * val_correct / val_total

    print(f"[Epoch {epoch+1}] Loss: {running_loss:.3f} | Train Acc: {acc:.2f}% | Val Acc: {val_acc:.2f}%")


[Epoch 1] Loss: 36.971 | Train Acc: 31.28% | Val Acc: 21.39%
[Epoch 2] Loss: 23.961 | Train Acc: 56.42% | Val Acc: 21.93%


In [41]:
import torch
import torchvision.transforms as transforms

# 테스트 이미지 경로
img_path = "/content/drive/MyDrive/김밥_Q5.jpg"

# 이미지 열기 + 전처리
image = Image.open(img_path).convert('RGB')
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])
input_tensor = transform(image).unsqueeze(0).to(device)  # 배치 차원 추가

# 예측
model.eval()
with torch.no_grad():
    output = model(input_tensor)
    _, predicted = torch.max(output, 1)

print("예측된 양 등급:", q_labels[predicted.item()])


예측된 양 등급: Q1
